# Diffusion mapping tutorial

Start by importing the required libraries and defining some settings:

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from ukat.data import fetch
from ukat.mapping.diffusion import ADC, DTI
from MDR import MDR, Tools
import ukat.moco.mdr_functions as mdr_functions

# Ensure figures are rendered in the notebook
%matplotlib inline

# Initialise output path for the Model-Driven Registration process
directory = 'diffusion_motion_correction_output'
os.makedirs(directory, exist_ok=True)
OUTPUT_DIR = os.path.join(os.getcwd(), directory)

In [6]:
import numpy as np
test_2d = np.ones((8,8,3))
test = [test_2d, test_2d, test_2d, test_2d, test_2d]
final = np.stack(test, axis=2)
print(np.shape(final))

(8, 8, 3, 5)


In [ ]:
# Fetch test data
pixel_array, affine, bvals, bvecs = fetch.dwi_philips()

In [ ]:
# Pre-processing as preparation for the Model-Driven Registration process
pixel_spacing = (np.linalg.norm(affine[:3, 1]), np.linalg.norm(affine[:3, 0]))
elastix_model_parameters = Tools.read_elastix_model_parameters("../ukat/moco/BSplines_DWI.txt")
list_input_parameters = [affine, bvals, None, False]
print(elastix_model_parameters)

In [ ]:
# Model-driven registration
moco_array = []
fit_array = []
for index in range(np.shape(pixel_array)[2]):
    pixel_array_2D = pixel_array[: , :, index, :]
    mdr_output_2D = MDR.model_driven_registration(pixel_array_2D, pixel_spacing, mdr_functions, list_input_parameters, elastix_model_parameters, precision=3, function='DWI_Moco', log=False)
    moco_array_2D = mdr_output_2D[0]
    fit_array_2D = mdr_output_2D[1]
    moco_array.append(moco_array_2D)
    fit_array.append(fit_array_2D)
moco_array = np.stack(moco_array, axis=2)
fit_array_2D = np.stack(fit_array_2D, axis=2)

In [ ]:
# Save motion corrected diffusion sequence to NIfTI
moco_nifti = nib.Nifti1Image(moco_array, affine=affine)
nib.save(moco_nifti, os.path.join(OUTPUT_DIR ,'DWI_motion_corrected.nii.gz'))

In [ ]:
# Generate a mask based on the intensity of the b0 volume. This will reduce computation times.
mask = pixel_array[..., 0] > 20000

# Calculate maps from the original diffusion sequence using ADC methods and save as niftis
adc_mapper = ADC(pixel_array, affine, bvals, mask=mask, ukrin_b=True)
adc_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_original', maps='all')

# Calculate maps from the original diffusion sequence using DTI methods and save as niftis
dti_mapper = DTI(pixel_array, affine, bvals, bvecs, mask=mask, ukrin_b=True)
dti_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_original', maps=['md', 'fa'])

# Calculate maps from the motion corrected diffusion sequence using ADC methods and save as niftis
adc_mapper = ADC(moco_array, affine, bvals, mask=mask, ukrin_b=True)
adc_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_moco', maps='all')

# Calculate maps from the motion corrected diffusion sequence using DTI methods and save as niftis
dti_mapper = DTI(moco_array, affine, bvals, bvecs, mask=mask, ukrin_b=True)
dti_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_moco', maps=['md', 'fa'])

In [ ]:
# Display the central slice of each map
fig, ax = plt.subplots(2, 2, figsize=(8, 6))

# Display a central slice of the ADC map
im = ax[0, 0].imshow(np.rot90(adc_mapper.adc[:, :, adc_mapper.shape[2]//2]), cmap='inferno', clim=(0.001, 0.003))
cb = fig.colorbar(im, ax=ax[0, 0])
cb.set_label('ADC ($mm^2/s$)')
ax[0, 0].axis('off')

# Display a central slice of the MD map
im = ax[0, 1].imshow(np.rot90(dti_mapper.md[:, :, dti_mapper.shape[2]//2]), cmap='inferno', clim=(0.001, 0.003))
cb = fig.colorbar(im, ax=ax[0, 1])
cb.set_label('MD ($mm^2/s$)')
ax[0, 1].axis('off')

# Display a central slice of the FA map
im = ax[1, 0].imshow(np.rot90(dti_mapper.fa[:, :, dti_mapper.shape[2]//2]), cmap='viridis', clim=(0.1, 0.8))
cb = fig.colorbar(im, ax=ax[1, 0])
cb.set_label('FA')
ax[1, 0].axis('off')

# Display a central slice of the color FA map
im = ax[1, 1].imshow(np.rot90(dti_mapper.color_fa[:, :, dti_mapper.shape[2]//2, :]))
ax[1, 1].axis('off')